In [1]:
from IPython.display import clear_output
import numpy as np
import pandas as pd
import random
import pygame
pygame.init()

pygame 2.3.0 (SDL 2.24.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


(5, 0)

# General

### Piece Representation

In [2]:
class Piece:
    def __init__(self, name, color,  value = 0):
        self.name = name
        self.color = color
        self.moved = False # used for piece history for castling / en-passant
        
    def __str__(self):
        if len(self.name) > 1:
            return str(self.color)[0] + str(self.name)[0] 
        else:
            return '  '

### Piece Images / Size Specs

In [3]:
WIDTH = 640
HEIGHT = 640
SIZE = WIDTH // 8
IMAGES = {}
pieces = ['white_pawn', 'white_knight', 'white_bishop', 'white_rook', 'white_queen', 'white_king', 
          'black_pawn', 'black_knight', 'black_bishop', 'black_rook', 'black_queen', 'black_king']
for piece in pieces:
    IMAGES[piece] = pygame.transform.scale(pygame.image.load(f"{piece}.png"), (SIZE, SIZE))

In [4]:
color_code = {'w': 'white', 'b': 'black'}
piece_codes = {
                "r": Piece("rook", "black"),
                "n": Piece("knight", "black"),
                "b": Piece("bishop", "black"),
                "q": Piece("queen", "black"),
                "k": Piece("king", "black"),
                "p": Piece("pawn", "black"),
                "R": Piece("rook", "white"),
                "N": Piece("knight", "white"),
                "B": Piece("bishop", "white"),
                "Q": Piece("queen", "white"),
                "K": Piece("king", "white"),
                "P": Piece("pawn", "white"),
              }

### Game Representation

In [5]:
class Game():
    def __init__(self):
        self.board = np.empty((8, 8), dtype = Piece)
        self.turn = 'white'
        self.selected_piece = None
        self.initial_row, self.initial_col = None, None
        self.valid_moves = []
        self.final_row, self.final_col = None, None
    
    def setup_board(self):
        # white pieces
        self.board[7][0] = Piece('rook', 'white')
        self.board[7][1] = Piece('knight', 'white')
        self.board[7][2] = Piece('bishop', 'white')
        self.board[7][3] = Piece('queen', 'white')
        self.board[7][4] = Piece('king', 'white')
        self.board[7][5] = Piece('bishop', 'white')
        self.board[7][6] = Piece('knight', 'white')
        self.board[7][7] = Piece('rook', 'white')
        for i in range(0, 8):
            self.board[6][i] = Piece('pawn', 'white')
        # black pieces
        self.board[0][0] = Piece('rook', 'black')
        self.board[0][1] = Piece('knight', 'black')
        self.board[0][2] = Piece('bishop', 'black')
        self.board[0][3] = Piece('queen', 'black')
        self.board[0][4] = Piece('king', 'black')
        self.board[0][5] = Piece('bishop', 'black')
        self.board[0][6] = Piece('knight', 'black')
        self.board[0][7] = Piece('rook', 'black')
        for i in range(0, 8):
            self.board[1][i] = Piece('pawn', 'black')
        # empty squares
        for x in range(2, 6):
            for y in range(0, 8):
                self.board[x][y] = Piece('', '')
    
    def setup_custom(self, fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR"):
        row = 0
        col = 0
        for char in fen:
            if char == '/':
                row += 1
                col = 0
            elif char.isnumeric():
                for i in range(int(char)):
                    self.board[row][col] = Piece('', '')
                    col += 1
            else:
                self.board[row][col] = piece_codes[char]
                col += 1

### Board On Screen

In [6]:
def draw_board(screen, game):
    for row in range(8):
        for col in range(8):
            if (row + col) % 2 == 0:
                pygame.draw.rect(screen, pygame.Color((238, 238, 210)), pygame.Rect(col * SIZE, row * SIZE, SIZE, SIZE))
            else:
                pygame.draw.rect(screen, pygame.Color((118, 150, 86)), pygame.Rect(col * SIZE, row * SIZE, SIZE, SIZE))
            piece = game.board[row][col]
            if piece.name != '':
                screen.blit(IMAGES[f"{piece.color}_{piece.name}"], pygame.Rect(col * SIZE, row * SIZE, SIZE, SIZE))

### FEN

In [7]:
def generate_fen(game):
    fen = ''
    empty_squares = 0
    for row in range(8):
        for col in range(8):
            if game.board[row][col].name == '':
                empty_squares += 1
            else:
                if empty_squares > 0:
                    fen += str(empty_squares)
                    empty_squares = 0
                if game.board[row][col].color == 'white':
                    if game.board[row][col].name == 'knight':
                        fen += 'N'
                    else:
                        fen += game.board[row][col].name[0].upper()
                else:
                    if game.board[row][col].name == 'knight':
                        fen += 'n'
                    else:
                        fen += game.board[row][col].name[0].lower()
        if empty_squares > 0:
            fen += str(empty_squares)
            empty_squares = 0
        fen += '/'
    return fen[:-1]

### Piece Movement Rules

In [8]:
def valid_move(game):
    
    if game.initial_row == game.final_row and game.initial_col == game.final_col: # starting square same as landing square
        return False
    if game.selected_piece.color == game.board[game.final_row][game.final_col].color: # capture your own piece
        return False
    if game.board[game.final_row][game.final_col].name == 'king': # capture king
        return False
    
    if game.selected_piece.name == 'pawn':
        direction = 1
        if game.selected_piece.color == 'white':
            direction = -1
        if game.final_col == game.initial_col: # no capture
            if game.final_row - game.initial_row == direction:
                if game.board[game.final_row][game.final_col].name == '':
                    return True
            if game.final_row - game.initial_row == 2 * direction:
                if game.initial_row == 1 or game.initial_row == 6:
                    if game.board[game.final_row - direction][game.final_col].name == game.board[game.final_row][game.final_col].name == '':
                        game.selected_piece.moved = True
                        return True
        elif abs(game.final_col - game.initial_col) == 1: # capture
            if game.final_row - game.initial_row == direction:
                if game.board[game.final_row][game.final_col].name != '':
                    return True
        return False
    
    if game.selected_piece.name == 'knight':
        row_diff = abs(game.final_row - game.initial_row)
        col_diff = abs(game.final_col - game.initial_col)
        if (row_diff == 2 and col_diff == 1) or (row_diff == 1 and col_diff == 2):
            return True
        return False
    
    elif game.selected_piece.name == 'bishop':
        if abs(game.initial_row - game.final_row) != abs(game.initial_col - game.final_col): # bishop needs to move on same diagonal
            return False
        
        # determine on which diagonal the bishop is moving
        row_step = -1
        col_step = -1
        if game.final_row > game.initial_row:
            row_step = 1
        if game.final_col > game.initial_col:
            col_step = 1
            
        # no piece on same diagonal between start and end
        row = game.initial_row + row_step
        col = game.initial_col + col_step
        while row != game.final_row and col != game.final_col:
            if game.board[row][col].name != '':
                return False
            row += row_step
            col += col_step
        return True
    
    elif game.selected_piece.name == 'rook':
        if game.initial_row != game.final_row and game.initial_col != game.final_col: # rook needs to move on same file or rank
            return False
        elif game.initial_row == game.final_row: # no piece on same rank between start and end
            start_col = min(game.initial_col, game.final_col)
            end_col = max(game.initial_col, game.final_col)
            for col in range(start_col + 1, end_col):
                if game.board[game.final_row][col].name != '':
                    return False
        else: # no piece on same file between start and end
            start_row = min(game.initial_row, game.final_row)
            end_row = max(game.initial_row, game.final_row)
            for row in range(start_row + 1, end_row):
                if game.board[row][game.final_col].name != '':
                    return False
        game.selected_piece.moved = True
        return True
    
    elif game.selected_piece.name == 'queen':
        if abs(game.initial_row - game.final_row) == abs(game.initial_col - game.final_col): # diagonal move - bishop rules
            row_step = -1
            col_step = -1
            if game.final_row > game.initial_row:
                row_step = 1
            if game.final_col > game.initial_col:
                col_step = 1
            row = game.initial_row + row_step
            col = game.initial_col + col_step
            while row != game.final_row and col != game.final_col:
                if game.board[row][col].name != '':
                    return False
                row += row_step
                col += col_step
            return True
        
        elif game.initial_row == game.final_row or game.initial_col == game.final_col: # straight move - rook rules
            if game.initial_row == game.final_row: # no piece on same rank between start and end
                start_col = min(game.initial_col, game.final_col)
                end_col = max(game.initial_col, game.final_col)
                for col in range(start_col + 1, end_col):
                    if game.board[game.final_row][col].name != '':
                        return False
            else: # no piece on same file between start and end
                start_row = min(game.initial_row, game.final_row)
                end_row = max(game.initial_row, game.final_row)
                for row in range(start_row + 1, end_row):
                    if game.board[row][game.final_col].name != '':
                        return False
            return True
        
    elif game.selected_piece.name == 'king':
        if abs(game.final_row - game.initial_row) <= 1 and abs(game.final_col - game.initial_col) <= 1:
            game.selected_piece.moved = True
            return True
        return False

### All Valid Moves

In [9]:
def get_valid_moves(game):
    for row in range(8):
        for col in range(8):
            game.final_row, game.final_col = row, col
            if valid_move(game):
                game.valid_moves.append((row, col))
    game.final_row, game.final_col = None, None

# Game Software

### Making Moves

In [10]:
def normal_move(screen, game):
    mouse_pos = pygame.mouse.get_pos()
    for row in range(8):
        for col in range(8):
            if pygame.Rect(col * SIZE, row * SIZE, SIZE, SIZE).collidepoint(mouse_pos): # if cursor is over square, outline square
                pygame.draw.rect(screen, pygame.Color((255, 255, 0)), pygame.Rect(col * SIZE, row * SIZE, SIZE, SIZE), 2)
                if pygame.mouse.get_pressed()[0] and game.selected_piece is None: # if mouse is pressed over valid piece, select piece if no other piece currently selected
                    if game.board[row][col].color == game.turn:
                        game.selected_piece = game.board[row][col]
                        game.initial_row, game.initial_col = row, col
                        if game.valid_moves == []:
                            get_valid_moves(game)
                elif not pygame.mouse.get_pressed()[0] and game.selected_piece is not None and game.final_row is None: # if pressed mouse is released, drop that piece on that square if this is a valid move
                    game.valid_moves = []
                    game.final_row, game.final_col = row, col
                    if valid_move(game):
                        game.board[game.initial_row][game.initial_col] = Piece('', '')
                        game.board[row][col] = game.selected_piece
                    else:
                        game.board[game.initial_row][game.initial_col] = game.selected_piece
                        game.selected_piece, game.initial_row, game.initial_col, game.final_row, game.final_col = None, None, None, None, None
    if game.selected_piece is not None and game.final_row is None: # if piece is selected, drag piece along with cursor and circle all possible landing squares
        for row, col in game.valid_moves:
            pygame.draw.circle(screen, (144, 172, 124), (col * SIZE + SIZE / 2, row * SIZE + SIZE / 2), 10)
        screen.blit(IMAGES[f"{game.selected_piece.color}_{game.selected_piece.name}"], (mouse_pos[0] - SIZE // 2, mouse_pos[1] - SIZE // 2))
    elif game.selected_piece is not None: # once move is completed, reset attributes and opponent's turn
        game.selected_piece, game.initial_row, game.initial_col, game.final_row, game.final_col = None, None, None, None, None
        if game.turn == 'white': # add in a checkmate check
            game.turn = 'black'
        else:
            game.turn = 'white'

### On Screen Visualization

In [11]:
def game():
    game = Game()
    game.setup_board()
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
        draw_board(screen, game)
        normal_move(screen, game)
        pygame.display.flip()

# Puzzle Software

### Importing Puzzle Data

In [12]:
import ast
df = pd.read_csv("puzzles.csv")
df.drop('Unnamed: 0', axis = 1, inplace = True)
df['positions'] = df['positions'].apply(ast.literal_eval) # removes quotation around list; e.g. "[1, 2, 3]" --> [1, 2, 3]
df = df.sample(frac=1).reset_index(drop=True)

### Making Moves

In [13]:
def puzzle_move(screen, game, fen_list):
    mouse_pos = pygame.mouse.get_pos()
    for row in range(8):
        for col in range(8):
            if pygame.Rect(col * SIZE, row * SIZE, SIZE, SIZE).collidepoint(mouse_pos): # if cursor is over square, outline square
                pygame.draw.rect(screen, pygame.Color((255, 255, 0)), pygame.Rect(col * SIZE, row * SIZE, SIZE, SIZE), 2)
                if pygame.mouse.get_pressed()[0] and game.selected_piece is None: # if mouse is pressed over valid piece, select piece if no other piece currently selected
                    if game.board[row][col].color == game.turn:
                        game.selected_piece = game.board[row][col]
                        game.initial_row, game.initial_col = row, col
                        if game.valid_moves == []:
                            get_valid_moves(game)
                elif not pygame.mouse.get_pressed()[0] and game.selected_piece is not None and game.final_row is None: # if pressed mouse is released, drop that piece on that square if this is a valid move
                    game.valid_moves = []
                    game.final_row, game.final_col = row, col
                    if valid_move(game):
                        game.board[game.initial_row][game.initial_col] = Piece('', '')
                        game.board[row][col] = game.selected_piece
                    else:
                        game.board[game.initial_row][game.initial_col] = game.selected_piece
                        game.selected_piece, game.initial_row, game.initial_col, game.final_row, game.final_col = None, None, None, None, None
    if game.selected_piece is not None and game.final_row is None: # if piece is selected, drag piece along with cursor and circle all possible landing squares
        for row, col in game.valid_moves:
            pygame.draw.circle(screen, (144, 172, 124), (col * SIZE + SIZE / 2, row * SIZE + SIZE / 2), 10)
        screen.blit(IMAGES[f"{game.selected_piece.color}_{game.selected_piece.name}"], (mouse_pos[0] - SIZE // 2, mouse_pos[1] - SIZE // 2))
    elif game.selected_piece is not None: # once move is completed, reset attributes and opponent's turn
        game.selected_piece, game.initial_row, game.initial_col, game.final_row, game.final_col = None, None, None, None, None
        if generate_fen(game) not in fen_list:
            return False
        elif generate_fen(game) == fen_list[-1]:
            return True
        elif fen_list.index(generate_fen(game)) % 2 == 1:
            game.setup_custom(fen_list[fen_list.index(generate_fen(game)) + 1])

### On Screen Visualization

In [14]:
def puzzle_display(color, positions):
    game = Game()
    game.turn = color
    game.setup_custom(positions[0])
    screen = pygame.display.set_mode((WIDTH, HEIGHT))
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
                pygame.quit()
        draw_board(screen, game)
        status = puzzle_move(screen, game, positions)
        if status == False:
            running = False
            pygame.quit()
            return True
        elif status:
            running = False
            pygame.quit()
        else:
            pygame.display.flip()

### Generating Puzzle Feed

In [15]:
def puzzle_feed():
    count = 0
    rating = int(input("Enter Initial Rating: "))
    clear_output()
    while True:
        curr_df = df[(df['rating'] > rating - 150) & (df['rating'] < rating + 150)]
        index = random.choice(curr_df.index)
        puzzle = curr_df.loc[index]
        color = puzzle['color']
        failed = puzzle_display(color, puzzle['positions'])
        if failed:
            print(f"The last puzzle was rated {puzzle['rating']}")
            print(f"You Correctly Solved {count} Puzzles")
            break
        count += 1
        rating += abs(rating - puzzle['rating']) / 2

In [16]:
if __name__ == "__main__":
    # game()
    puzzle_feed()

The last puzzle was rated 2562
You Correctly Solved 0 Puzzles
